In [1]:
import os
import warnings

warnings.filterwarnings("ignore")
if os.getcwd()[-8:]=='examples':
    os.chdir('..')
    
os.environ["CUDA_VISIBLE_DEVICES"] = "3" # Possible GPUS

In [2]:
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image

from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
import torchdefenses as td
import torchdefenses.trainer as tr
from torchdefenses.transforms.cutout import Cutout

import src
from src.utils import *

### Settings

In [3]:
#### FOR TRAINING
EPOCHS = 100
BATCH_SIZE = 64

#### FOR MODELING
MODEL_NAME = "WRN16-4_WS"
DATA = "CIFAR10"
MEAN = [0.4930, 0.4875, 0.4548]
STD = [0.2475, 0.2445, 0.2637]
N_CLASSES = 10
LR = 0.1
RHO = 0.1

#### PATH SAVING
PATH = "./saved/"
NAME = "cifar-10-warmup"
SAVE_PATH = PATH + NAME

### Data loader

In [4]:
## FOR EDM
# images = torch.load('./data/cifar-10-edm/cifar10_data_sampled.pt').type('torch.FloatTensor')
# labels = torch.load('./data/cifar-10-edm/cifar10_data_sampled_labels.pt').type('torch.LongTensor')

# train_transform  = transforms.Compose([
#                     transforms.RandomCrop(32, padding=4),
#                     transforms.RandomHorizontalFlip(),
#                     transforms.Normalize(MEAN, STD),
#                     Cutout(),
#                    ])

## FOR DG
data = np.load('./data/cifar-10-edm/cifar10_data_sampled_weight0.npz')
images = data['samples'][-50000:]
images = [Image.fromarray(np.array(img)) for img in images]
labels = torch.Tensor(data['labels'][-50000:]).type('torch.LongTensor')
labels = torch.argmax(labels, dim=1)

train_transform  = transforms.Compose([
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize(MEAN, STD),
                    Cutout(),
                   ])

In [ ]:
dataset = PtDataset(images, labels, transform=train_transform)
public_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, 
                           drop_last=False, num_workers=16, pin_memory=True)

data = td.Datasets(data_name=DATA,
                   train_transform = transforms.Compose([
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize(MEAN, STD),
                    Cutout(),
                   ]),
                   val_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize(MEAN, STD),
                   ]),
                   test_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize(MEAN, STD),
                   ]))
_, test_loader = data.get_loader(batch_size=BATCH_SIZE, drop_last_train=False, num_workers=16)

### Model & Optimizer

In [7]:
#### Load model
model = src.utils.load_model(model_name=MODEL_NAME, n_classes=N_CLASSES).cuda() # Load model
model = ModuleValidator.fix(model)
set_groupnorm_num_groups(model, num_groups=16)

ModuleValidator.validate(model, strict=False)

pytorch_total_params = sum(p.numel() for p in model.parameters())

print("model params: {:.4f}M".format(pytorch_total_params/1000000))

rmodel = td.RobModel(model, n_classes=N_CLASSES, normalization_used={'mean':MEAN, 'std':STD})

WRN16-4_WS is loaded.
model params: 2.7489M


### Start Training

In [8]:
trainer = tr.Standard(rmodel)
trainer.setup(optimizer=f"SGD(lr={LR}, momentum=0.9, weight_decay=0.0005)",
              scheduler="Cosine", scheduler_type="Epoch",
              minimizer=f"SAM(rho={RHO})",
              n_epochs=EPOCHS, n_iters=len(public_loader)
             )
trainer.record_rob(public_loader, test_loader, eps=None)
trainer.fit(train_loader=public_loader, n_epochs=EPOCHS,
            save_path=SAVE_PATH, save_best={"Clean(Val)":"HBO"},
            save_type=None, save_overwrite=True, record_type="Epoch")

[Standard]
Training Information.
-Epochs: 100
-Optimizer: SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    initial_lr: 0.1
    lr: 0.1
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0005
)
-Scheduler: <torch.optim.lr_scheduler.CosineAnnealingLR object at 0x7fe3b368b820>
-Minmizer: <torchdefenses.optim.minimizer.SAM object at 0x7fe39e7555e0>
-Save Path: ./saved/cifar-10-warmup
-Save Type: None
-Record Type: Epoch
-Device: cuda:0
-----------------------------------------------------------------------
Epoch   CALoss   CALoss_1   Clean(Tr)   Clean(Val)   lr       s/it     
1       1.1194   1.2725     51.0660     46.4900      0.1000   0.0814   
-----------------------------------------------------------------------
2       0.9260   1.2273     71.0780     60.6300      0.1000   0.0783   
-----------------------------------------------------------------------
3       0.4142   0.5942     83.8560     66.0300      0.0999   0.0

### Evaluation

In [9]:
rmodel.load_dict(PATH+NAME+'/last.pth')
rmodel.eval_accuracy(test_loader)

Model loaded.
Record Info:
OrderedDict([('Epoch', 100), ('Iter', 782), ('CALoss', 0.003191609401255846), ('CALoss_1', 0.12312682718038559), ('Clean(Tr)', 99.948), ('Clean(Val)', 80.76), ('lr', 2.4671981713420017e-05), ('s/it', 0.0778921189258312)])


80.76

In [10]:
rmodel.load_dict(PATH+NAME+'/best.pth')
rmodel.eval_accuracy(test_loader)

Model loaded.
Record Info:
OrderedDict([('Epoch', 99), ('Iter', 782), ('CALoss', 0.005149493459612131), ('CALoss_1', 0.09331594407558441), ('Clean(Tr)', 99.936), ('Clean(Val)', 80.77), ('lr', 9.866357858642213e-05), ('s/it', 0.07660343606138108)])


80.77